In [4]:
!pip install keras_tuner


[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: C:\Users\Matthew Wijaya\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [2]:
from pymfe.mfe import MFE
import pandas as pd
import tensorflow as tf
import numpy as np
import os
import optuna
from tensorflow.keras.models import Sequential
from keras.src.layers import Dense
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
import matplotlib.pyplot as plt
from scipy import stats
import ast
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
import keras_tuner
import shutil
import random
import time
import itertools

C:\Users\Matthew Wijaya\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [7]:
data_folder = r"D:\MATTHEW\UROPS\Datasets\All Datasets\All Formatted Valid Datasets"  # Change this
train_folder = r"D:\MATTHEW\UROPS\Datasets\All Datasets\All Formatted Valid Datasets\Train Datasets"
test_folder = r"D:\MATTHEW\UROPS\Datasets\All Datasets\All Formatted Valid Datasets\Test Datasets"

# Create train/test folders if they don't exist
os.makedirs(train_folder, exist_ok=True)
os.makedirs(test_folder, exist_ok=True)

# Get all dataset files (assuming CSV, modify for other formats)
dataset_files = [f for f in os.listdir(data_folder) if f.endswith(".csv")]

# Shuffle datasets and split (80 for train, 20 for test)
random.seed(42)  # For reproducibility
random.shuffle(dataset_files)

train_files = dataset_files[:80]  # First 80 datasets
test_files = dataset_files[80:]   # Remaining 20 datasets

# Move files to respective folders
for file in train_files:
    shutil.move(os.path.join(data_folder, file), os.path.join(train_folder, file))

for file in test_files:
    shutil.move(os.path.join(data_folder, file), os.path.join(test_folder, file))

print(f"✅ Split complete! Train: {len(train_files)}, Test: {len(test_files)}")

✅ Split complete! Train: 80, Test: 20


In [22]:
cols_with_na = mfe_df.columns[mfe_df.isna().any()].tolist()
print(cols_with_na)

['cor.mean', 'cor.sd', 'cov.sd', 'g_mean.mean', 'g_mean.sd', 'h_mean.mean', 'h_mean.sd', 'kurtosis.mean', 'kurtosis.sd', 'skewness.mean', 'skewness.sd']


In [29]:
mfe_df = mfe_df.drop(columns=cols_with_na)

In [28]:
df = pd.read_csv(r'D:\MATTHEW\UROPS\Datasets\All Datasets\All Formatted Datasets\cnae9.csv')
df[df['V505'] == 1]

,V505,V552,V585,V520,V208,V7,V349,V369,V851,V559,class


In [31]:
mfe_df

,attr_conc.mean,attr_conc.sd,attr_ent.mean,attr_ent.sd,attr_to_inst,cat_to_num,class_conc.mean,class_conc.sd,class_ent,cov.mean,...,nr_cor_attr,nr_inst,nr_norm,nr_num,nr_outliers,ns_ratio,sparsity.mean,sparsity.sd,t_mean.mean,t_mean.sd
100_plants_margin.csv,0.046970,0.033535,3.335117,0.156850,0.006250,0.0,0.001916,0.000885,1.000000,1.147282e-04,...,0.222222,1600,0.0,10,10,282.039083,0.021805,0.008882,0.015267,0.006479
100_plants_shape.csv,0.145505,0.087566,3.459307,0.000056,0.006250,0.0,0.003121,0.001214,1.000000,6.415149e-08,...,0.777778,1600,0.0,10,10,150.549493,0.000703,0.000045,0.000548,0.000090
100_plants_texture.csv,0.059551,0.065971,2.650849,0.442223,0.006258,0.0,0.002163,0.001136,1.000000,2.566332e-04,...,0.044444,1598,0.0,10,10,278.432591,0.007767,0.003522,0.006863,0.003775
amazon_review.csv,0.002245,0.003200,0.604897,0.797588,0.006667,0.0,0.001762,0.002069,1.000000,6.921859e-03,...,0.000000,1500,0.0,10,10,327.872083,0.250814,0.129059,0.166889,0.277774
analcatdata_authorship.csv,0.014975,0.008193,2.843746,0.181160,0.014245,0.0,0.022209,0.017412,1.000000,4.684453e+00,...,0.000000,702,0.0,10,10,25.787208,0.045674,0.022388,7.219194,4.755125
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
transfusion.csv,0.222062,0.366012,2.882541,0.268578,0.005348,0.0,0.010159,0.008808,0.791645,5.568267e+03,...,0.500000,748,0.0,4,3,72.116525,0.025118,0.009126,267.449444,506.647734
tumors_C.csv,0.074376,0.043501,1.584963,0.000000,0.238095,0.0,0.031973,0.042307,1.000000,8.839196e+04,...,0.000000,42,1.0,10,8,31.732803,0.000804,0.000845,963.080769,1941.451650
vowel.csv,0.037007,0.041231,2.952574,0.687268,0.011111,0.0,0.004018,0.003940,1.000000,1.010581e-01,...,0.066667,900,0.0,10,4,123.730938,0.050147,0.157867,-0.045583,1.299892
wall_robot_navigation.csv,0.101068,0.058896,3.713979,0.289277,0.002062,0.0,0.012968,0.005302,1.000000,2.842256e-01,...,0.022222,4850,0.0,10,6,27.423493,0.000420,0.000141,1.866406,0.816031


In [16]:
results_df = pd.read_csv(r"D:\MATTHEW\UROPS\Datasets\All Datasets\All Datasets Results.csv", index_col=0)
results_complete_df = pd.read_csv(r"D:\MATTHEW\UROPS\Datasets\All Datasets\All Datasets Results Complete.csv", index_col=0).reindex(results_df.index)
encoded_df = pd.read_csv(r"D:\MATTHEW\UROPS\Datasets\All Datasets\All Datasets Encoded.csv", index_col=0).reindex(results_df.index)
mfe_df = pd.read_csv(r"D:\MATTHEW\UROPS\Datasets\All Datasets\All Datasets Common Meta-Features.csv", index_col=0).reindex(results_df.index)

In [33]:
mfe_df

,attr_conc.mean,attr_conc.sd,attr_ent.mean,attr_ent.sd,attr_to_inst,cat_to_num,class_conc.mean,class_conc.sd,class_ent,cov.mean,...,nr_cor_attr,nr_inst,nr_norm,nr_num,nr_outliers,ns_ratio,sparsity.mean,sparsity.sd,t_mean.mean,t_mean.sd
100_plants_margin.csv,0.046970,0.033535,3.335117,0.156850,0.006250,0.0,0.001916,0.000885,1.000000,1.147282e-04,...,0.222222,1600,0.0,10,10,282.039083,0.021805,0.008882,0.015267,0.006479
100_plants_shape.csv,0.145505,0.087566,3.459307,0.000056,0.006250,0.0,0.003121,0.001214,1.000000,6.415149e-08,...,0.777778,1600,0.0,10,10,150.549493,0.000703,0.000045,0.000548,0.000090
100_plants_texture.csv,0.059551,0.065971,2.650849,0.442223,0.006258,0.0,0.002163,0.001136,1.000000,2.566332e-04,...,0.044444,1598,0.0,10,10,278.432591,0.007767,0.003522,0.006863,0.003775
amazon_review.csv,0.002245,0.003200,0.604897,0.797588,0.006667,0.0,0.001762,0.002069,1.000000,6.921859e-03,...,0.000000,1500,0.0,10,10,327.872083,0.250814,0.129059,0.166889,0.277774
analcatdata_authorship.csv,0.014975,0.008193,2.843746,0.181160,0.014245,0.0,0.022209,0.017412,1.000000,4.684453e+00,...,0.000000,702,0.0,10,10,25.787208,0.045674,0.022388,7.219194,4.755125
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
transfusion.csv,0.222062,0.366012,2.882541,0.268578,0.005348,0.0,0.010159,0.008808,0.791645,5.568267e+03,...,0.500000,748,0.0,4,3,72.116525,0.025118,0.009126,267.449444,506.647734
tumors_C.csv,0.074376,0.043501,1.584963,0.000000,0.238095,0.0,0.031973,0.042307,1.000000,8.839196e+04,...,0.000000,42,1.0,10,8,31.732803,0.000804,0.000845,963.080769,1941.451650
vowel.csv,0.037007,0.041231,2.952574,0.687268,0.011111,0.0,0.004018,0.003940,1.000000,1.010581e-01,...,0.066667,900,0.0,10,4,123.730938,0.050147,0.157867,-0.045583,1.299892
wall_robot_navigation.csv,0.101068,0.058896,3.713979,0.289277,0.002062,0.0,0.012968,0.005302,1.000000,2.842256e-01,...,0.022222,4850,0.0,10,6,27.423493,0.000420,0.000141,1.866406,0.816031


In [18]:
# Mode Model
X_mode = results_df
y_mode = results_df.idxmax(axis=1)
X_mode_train, X_mode_test, y_mode_train, y_mode_test = train_test_split(X_mode, y_mode, test_size = 0.2)
mode_col = X_mode_train.sum().idxmax()
print(mode_col)
count = 0
for prediction in y_mode_test:
    if prediction == mode_col:
        count+= 1
print(count / len(y_mode_test))


13
0.6


In [ ]:
#KNN Landmarkers
X_landmark = results_df
y_landmark = results_df.idxmax(axis=1)
X_landmark_train, X_landmark_test, y_landmark_train, y_landmark_test = train_test_split(X_landmark, y_landmark, test_size = 0.2)
knn_landmark = KNeighborsClassifier(n_neighbors=3)
knn_landmark.fit(X_landmark_train, y_landmark_train)

y_knn_landmark_pred = knn_landmark.predict(X_landmark_test)

print("Accuracy:", accuracy_score(y_landmark_test, y_knn_landmark_pred))

Accuracy: 0.5


In [ ]:
#KNN MFE
X_mfe = mfe_df
y_mfe = results_df.idxmax(axis=1)
X_mfe_train, X_mfe_test, y_mfe_train, y_mfe_test = train_test_split(X_mfe, y_mfe, test_size = 0.2)
knn_mfe = KNeighborsClassifier(n_neighbors=3)
knn_mfe.fit(X_mfe_train, y_mfe_train)

y_knn_mfe_pred = knn_mfe.predict(X_mfe_test)

print("Accuracy:", accuracy_score(y_mfe_test, y_knn_mfe_pred))

Accuracy: 0.55


In [37]:
#SVM Landmarkers
svm_landmark = SVC()
svm_landmark.fit(X_landmark_train, y_landmark_train)

y_svm_landmark_pred = svm_landmark.predict(X_landmark_test)

print("Accuracy:", accuracy_score(y_landmark_test, y_svm_landmark_pred))

Accuracy: 0.5


In [45]:
#SVM MFE
svm_mfe = SVC()
svm_mfe.fit(X_mfe_train, y_mfe_train)

y_svm_mfe_pred = svm_mfe.predict(X_mfe_test)

print('Accuracy:', accuracy_score(y_mfe_test, y_svm_mfe_pred))

Accuracy: 0.6


In [41]:
#Decision Tree Landmarkers
dt_landmark = DecisionTreeClassifier()
dt_landmark.fit(X_landmark_train, y_landmark_train)

y_dt_landmark_pred = dt_landmark.predict(X_landmark_test)

print("Accuracy:", accuracy_score(y_landmark_test, y_dt_landmark_pred))

Accuracy: 0.5


In [44]:
#Decision Tree MFE
dt_mfe = DecisionTreeClassifier()
dt_mfe.fit(X_mfe_train, y_mfe_train)

y_dt_mfe_pred = dt_mfe.predict(X_mfe_test)

print('Accuracy:', accuracy_score(y_mfe_test, y_dt_mfe_pred))

Accuracy: 0.3


In [3]:
def build_model(hp):
    model = Sequential()
    # Tune the number of layers.
    for i in range(hp.Int("num_layers", 1, 3)):
        model.add(
            Dense(
                # Tune number of units separately.
                units=hp.Int(f"units_{i}", min_value=1, max_value=16, step=1),
                activation='relu',
            )
        )
    model.add(Dense(1, activation="sigmoid"))
    model.compile(
        optimizer=Adam(learning_rate=0.001),
        loss="binary_crossentropy",
        metrics=["accuracy"],
    )
    return model

In [17]:
gs = keras_tuner.GridSearch(hypermodel=build_model, objective='val_accuracy')
gs.search_space_summary()

Search space summary
Default search space size: 2
num_layers (Int)
{'default': None, 'conditions': [], 'min_value': 1, 'max_value': 3, 'step': 1, 'sampling': 'linear'}
units_0 (Int)
{'default': None, 'conditions': [], 'min_value': 1, 'max_value': 16, 'step': 1, 'sampling': 'linear'}


In [28]:
for col in df.columns:
    print(df[col].dtype)

float64
float64
int64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
int64


In [ ]:
df = pd.read_csv(r"D:\MATTHEW\UROPS\Datasets\All Datasets\All Formatted Valid Datasets\Test Datasets\Algerian_forest_fires_dataset_UPDATE.csv", dtype=str)
df = df.apply(pd.to_numeric, errors="coerce")
X = df.iloc[:,:-1].values
y = df.iloc[:,-1].values
gs = keras_tuner.GridSearch(hypermodel=build_model, objective='val_accuracy', directory=r'D:\MATTHEW\UROPS\Datasets\All Datasets', project_name = 'GridSearch')
kf = KFold(n_splits=10)
for train_index, val_index in kf.split(X):
    X_train, X_val = X[train_index], X[val_index]
    y_train, y_val = y[train_index], y[val_index]
    gs.search(X_train, y_train, validation_data=(X_val, y_val))

Trial 74 Complete [00h 00m 02s]
val_accuracy: 0.523809552192688

Best val_accuracy So Far: 0.523809552192688
Total elapsed time: 00h 02m 18s

Search: Running Trial #75

Value             |Best Value So Far |Hyperparameter
2                 |1                 |num_layers
4                 |2                 |units_0
11                |None              |units_1



In [31]:
models = gs.get_best_models(num_models=2)
best_model = models[0]
best_model.summary()

C:\Users\Matthew Wijaya\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\keras\src\saving\saving_lib.py:757: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 10 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


NotFoundError: NewRandomAccessFile failed to Create/Open: .\untitled_project\trial_0000\build_config.json : The system cannot find the file specified.
; No such file or directory

In [4]:
#Random Search
def random_search(dir):
    result = {}
    input_dir = dir + '\\' + 'Formatted Datasets'
    for file in os.listdir(input_dir):
    # for file in ["air_quality.csv"]:
        print(file)
        start = time.time()
        df = pd.read_csv(input_dir + '\\' + file, dtype=str)
        df = df.apply(pd.to_numeric, errors="coerce")
        X = df.iloc[:,:-1].values
        y = df.iloc[:,-1].values
        rs = keras_tuner.RandomSearch(hypermodel=build_model, objective='val_accuracy', directory=r'D:\MATTHEW\UROPS\Datasets\All Datasets\RandomSearch', project_name = file)
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
        rs.search(X_train, y_train, validation_data=(X_test, y_test))
        best_hp = rs.get_best_hyperparameters(1)
        # print(best_hp[0])
        
        accuracies = []
        kf = KFold(n_splits=10)
        for train_index, val_index in kf.split(X):
            X_train, X_val = X[train_index], X[val_index]
            y_train, y_val = y[train_index], y[val_index]
            best_model = build_model(best_hp[0])
            best_model.fit(X_train, y_train)
            y_pred_prob = best_model.predict(X_val)
            y_pred_labels = (y_pred_prob >= 0.5).astype(int)
            accuracies.append(accuracy_score(y_val, y_pred_labels))
        end = time.time()
        runtime = end - start
        num_layers = best_hp[0].get('num_layers')
        num_neurons = []
        for i in range(num_layers):
            num_neurons.append(best_hp[0].get(f'units_{i}'))
        result[file] = {'best_hp' : (num_layers, num_neurons), 'mean_accuracy' : np.mean(accuracies), 'runtime' : runtime}
    return result
    


    

In [41]:
rs.search_space_summary()

NameError: name 'rs' is not defined

In [ ]:
for i in range(5, 9):
    print(f'Batch Valid {i}')
    dir = r'D:\MATTHEW\UROPS\Datasets\All Datasets\Train Datasets\Train Batch ' + str(i)
    results = random_search(dir)
    results_df = pd.DataFrame(results).T
    results_df.to_csv(r'D:\MATTHEW\UROPS\Datasets\All Datasets\Random Search Results' + '\\' + f'Train Batch {i} Random Search Results.csv', index = False)

Trial 6 Complete [00h 00m 30s]
val_accuracy: 0.652092695236206

Best val_accuracy So Far: 0.6707772612571716
Total elapsed time: 00h 02m 53s

Search: Running Trial #7

Value             |Best Value So Far |Hyperparameter
2                 |1                 |num_layers
5                 |7                 |units_0
11                |11                |units_1



Exception ignored in: <function WeakKeyDictionary.__init__.<locals>.remove at 0x000001C62544C5E0>
Traceback (most recent call last):
  File "C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.11_3.11.2544.0_x64__qbz5n2kfra8p0\Lib\weakref.py", line 370, in remove
    self = selfref()
           ^^^^^^^^^
KeyboardInterrupt: 


In [6]:
i

5

In [ ]:
for i in range(1, 3):
    print(f'Test Batch {i}')
    dir = r'D:\MATTHEW\UROPS\Datasets\All Datasets\Test Datasets\Test Batch ' + str(i)
    results = random_search(dir)
    results_df = pd.DataFrame(results).T
    results_df.to_csv(r'D:\MATTHEW\UROPS\Datasets\All Datasets\Random Search Results' + '\\' + f'Test Batch {i} Random Search Results.csv', index = False)

In [5]:
pd.DataFrame(results).T

NameError: name 'results' is not defined

In [7]:
result_df = pd.DataFrame(result).T

In [8]:
result_df.to_csv(r'D:\MATTHEW\UROPS\Datasets\All Datasets\Random Search Test Results.csv')

In [ ]:
result_df

,best_hp,mean_accuracy,runtime
air_quality.csv,"(2, [8, 4])",0.515,14.021838
Algerian_forest_fires_dataset_UPDATE.csv,"(2, [4, 2])",0.466667,11.963502
Banknote.csv,"(1, [16])",0.659016,11.12095
breast-cancer-wisconsin.csv,"(1, [3])",0.377748,11.044911
cancer.csv,"(2, [2, 8])",0.419048,14.937386
Churn.csv,"(2, [16, 14])",0.34573,13.235838
dataset_37_diabetes.csv,"(1, [5])",0.649511,11.173062
dataset_53_heart-statlog.csv,"(1, [2])",0.4,12.006644
dataset_54_vehicle.csv,"(3, [9, 4, 7])",0.481913,34.506789
drug200.csv,"(2, [11, 2])",0.514286,24.542058


In [ ]:
#Grid Search
gs_result = {}
for file in os.listdir(r"D:\MATTHEW\UROPS\Datasets\All Datasets\All Formatted Valid Datasets"):
# for file in ["air_quality.csv"]:
    print(file)
    start = time.time()
    df = pd.read_csv(r"D:\MATTHEW\UROPS\Datasets\All Datasets\All Formatted Valid Datasets" + '\\' + file, dtype=str)
    df = df.apply(pd.to_numeric, errors="coerce")
    X = df.iloc[:,:-1].values
    y = df.iloc[:,-1].values
    gs = keras_tuner.GridSearch(hypermodel=build_model, objective='val_accuracy', directory=r'D:\MATTHEW\UROPS\Datasets\All Datasets\GridSearch', project_name = file)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
    gs.search(X_train, y_train, validation_data=(X_test, y_test))
    best_hp = gs.get_best_hyperparameters(1)
    # print(best_hp[0])
    
    accuracies = []
    kf = KFold(n_splits=10)
    for train_index, val_index in kf.split(X):
        X_train, X_val = X[train_index], X[val_index]
        y_train, y_val = y[train_index], y[val_index]
        best_model = build_model(best_hp[0])
        best_model.fit(X_train, y_train)
        y_pred_prob = best_model.predict(X_val)
        y_pred_labels = (y_pred_prob >= 0.5).astype(int)
        accuracies.append(accuracy_score(y_val, y_pred_labels))
    end = time.time()
    runtime = end - start
    num_layers = best_hp[0].get('num_layers')
    num_neurons = []
    for i in range(num_layers):
        num_neurons.append(best_hp[0].get(f'units_{i}'))
    gs_result[file] = {'best_hp' : (num_layers, num_neurons), 'mean_accuracy' : np.mean(accuracies), 'runtime' : runtime}
    cur_df = pd.DataFrame(gs_result[file])
    cur_df.to_csv(r"D:\MATTHEW\UROPS\Datasets\All Datasets\Grid Search Results" + '\\' + file)
    


    

Trial 2158 Complete [00h 03m 08s]
val_accuracy: 0.4871794879436493

Best val_accuracy So Far: 0.7435897588729858
Total elapsed time: 18h 05m 05s


In [ ]:
gs_result_df = pd.DataFrame(gs_result).T

In [ ]:
gs_result_df

In [ ]:
gs_result_df.to_csv(r'D:\MATTHEW\UROPS\Datasets\All Datasets\Grid Search Test Results.csv')

In [16]:
test = {'a' : [4, 8, 12, 16], 'b': [4, 8, 12, 16], 'c' : [4, 8, 12, 16]}
comb = list(itertools.product(*test.values()))

In [17]:
test_2 = {'a' : [4, 8, 12, 16], 'b': [4, 8, 12, 16], 'c' : [0]}
test_3 = {'a' : [4, 8, 12, 16], 'b': [0], 'c' : [0]}
comb_2 = list(itertools.product(*test_2.values()))
comb_3 = list(itertools.product(*test_3.values()))

In [21]:
combs = comb + comb_2 + comb_3

In [34]:
hp_set = []
for i in combs:
    count = 0
    for j in i:
        if j != 0:
            count+= 1
    cur_hp = (count, list(i))
    hp_set.append(cur_hp)

In [36]:
len(hp_set)

84

In [27]:
# Manual Grid Search

def grid_search(dir, hp_set):
    input_dir = dir + '\\' + 'Formatted Datasets'
    split_dir = dir + '\\' + 'Splitted Datasets'
    results = {}
    complete_results = {}
    runtimes = {}

    for file in os.listdir(input_dir):
        print(file)
        cur_result = {}
        cur_result_complete = {}
        for fold in range(1, 11):
            train_df = pd.read_csv(split_dir + '\\' + f'Train Fold {fold} ' + file)
            val_df = pd.read_csv(split_dir + '\\' + f'Val Fold {fold} ' + file)
            X_train = train_df.iloc[:, :-1].values  
            y_train = train_df.iloc[:, -1].values   
            X_val = val_df.iloc[:, :-1].values  
            y_val = val_df.iloc[:, -1].values
            for i in range(len(hp_set)):
                start = time.time()
                hp = hp_set[i]
                if i not in cur_result:
                    cur_result[i] = []
                if i not in cur_result_complete:
                    cur_result_complete[i] = []
                model = create_mlp(X_train, y_train, hp[0], hp[1])
                model.fit(X_train, y_train, epochs=10, batch_size=32, verbose=0)
                val_loss, val_accuracy = model.evaluate(X_val, y_val, verbose=0)
                cur_result[i].append(val_accuracy)
                cur_result_complete[i].append(val_accuracy)
                y_pred_train = model.predict(X_train).flatten()
                y_pred_val = model.predict(X_val).flatten()
                y_pred = list(y_pred_train) + list(y_pred_val)
                y = list(y_train) + list(y_val)
                pred_df = pd.DataFrame({'y' : y, 'y_pred' : y_pred})
                # print(pred_df)
                pred_df.to_csv(dir + '\\' + 'Complete Results' + '\\' + f'Complete Results Fold {fold} {hp_set[i]} ' + file, index=False)
                end = time.time()
                
                runtime = end - start
                if file + str(hp) not in runtimes:
                    runtimes[file + str(hp)] = runtime
                else:
                    runtimes[file + str(hp)] += runtime
        
        complete_results[file] = cur_result_complete
        print(complete_results)
        for i in cur_result:
            cur_result[i] = np.mean(cur_result[i])
        results[file] = cur_result
        
    
    return results, complete_results, runtimes


In [25]:
def make_splits(input_dir, split_dir):
    for file in os.listdir(input_dir):
        print(file)
        dataset = pd.read_csv(input_dir + '\\' + file)
        # print(dataset)
        X = dataset.iloc[:, :-1].values  # All columns except the last one for features
        y = dataset.iloc[:, -1].values   # Last column for target
        # print(labels)
        kf = KFold(n_splits=10, shuffle=True)
        for fold, (train_index, val_index) in enumerate(kf.split(X), start=1):
            dataset.iloc[train_index].to_csv(split_dir + '\\' + f'Train Fold {fold} ' + file, index=False)
            dataset.iloc[val_index].to_csv(split_dir + '\\' + f'Val Fold {fold} ' + file, index=False)

In [30]:
def create_mlp(X, y, num_layers, num_neurons):
    model = Sequential()

    model.add(Dense(num_neurons[0], input_dim = X.shape[1], activation = 'relu'))

    for i in range(1, num_layers):
        model.add(Dense(num_neurons[i], activation = 'relu'))

    model.add(Dense(1, activation='sigmoid'))

    model.compile(optimizer=Adam(learning_rate=0.001),
                  loss='binary_crossentropy',
                  metrics=['accuracy'])
    return model

In [ ]:
for i in [5]:
    print(f'Batch Valid {i}')
    dir = r'D:\MATTHEW\UROPS\Datasets\All Datasets\Batch Valid ' + str(i)
    make_splits(dir + r'\Formatted Datasets', dir + r'\Splitted Datasets')
    results, complete_results = hp_space_experiment_from_split(dir, test_hp_set)
    results_df = pd.DataFrame(results).T
    results_complete_df = pd.DataFrame(complete_results).T
    results_df.to_csv(dir + '\\' + f'Batch Valid {i} Results.csv')
    results_complete_df.to_csv(dir + '\\' + f'Batch Valid {i} Results Complete.csv')

In [37]:
make_splits(r'D:\MATTHEW\UROPS\Datasets\All Datasets\Test\Formatted Datasets', r'D:\MATTHEW\UROPS\Datasets\All Datasets\Test\Splitted Datasets')
test_result, test_result_complete, rt = grid_search(r"D:\MATTHEW\UROPS\Datasets\All Datasets\Test", hp_set)

Fertility.csv
Fertility.csv


C:\Users\Matthew Wijaya\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 183ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 143ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 252ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 127ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 116ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 204ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 66ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 161ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 161ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 120ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 135ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 74ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 81ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 102ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 66ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 83ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 215ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 82ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 101ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 91ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 208ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 74ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 197ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 183ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 115ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 140ms

C:\Users\Matthew Wijaya\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 170ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 69ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 197ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 183ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 68ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 89ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 82ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 200ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 139ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 121ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 170ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 169ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 160ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 237ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 112ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 277ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 134ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 79ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 89ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 229ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 117ms/st

...

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 228ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 248ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 90ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 119ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 212ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 65ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 220ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 82ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 78ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 203ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 90ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 86ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 73ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 158ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 115ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 70ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 148ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 72ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 66ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 119ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 89ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 73ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step
1/

C:\Users\Matthew Wijaya\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 97ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 108ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 92ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 81ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 127ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 141ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 133ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 97ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 91ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 100ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 279ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 111ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 132ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 97ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 78ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 154ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 128ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 189ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 64ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 65ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 66ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 100ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 135ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step

C:\Users\Matthew Wijaya\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 172ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 93ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 121ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 122ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 82ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 102ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 72ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 122ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 69ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 230ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 67ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 183ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 83ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 223ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 78ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 82ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 80ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 89ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 190ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 218ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 202ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 75ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 149ms/step


In [38]:
rt

{'Fertility.csv(3, [4, 4, 4])': 33.279576539993286,
 'Fertility.csv(3, [4, 4, 8])': 33.58117413520813,
 'Fertility.csv(3, [4, 4, 12])': 33.668102979660034,
 'Fertility.csv(3, [4, 4, 16])': 32.10842561721802,
 'Fertility.csv(3, [4, 8, 4])': 42.096259117126465,
 'Fertility.csv(3, [4, 8, 8])': 33.20982313156128,
 'Fertility.csv(3, [4, 8, 12])': 39.66255760192871,
 'Fertility.csv(3, [4, 8, 16])': 33.75223755836487,
 'Fertility.csv(3, [4, 12, 4])': 31.914332628250122,
 'Fertility.csv(3, [4, 12, 8])': 31.436097621917725,
 'Fertility.csv(3, [4, 12, 12])': 52.785398960113525,
 'Fertility.csv(3, [4, 12, 16])': 37.31790328025818,
 'Fertility.csv(3, [4, 16, 4])': 35.365193605422974,
 'Fertility.csv(3, [4, 16, 8])': 33.35034155845642,
 'Fertility.csv(3, [4, 16, 12])': 35.25629949569702,
 'Fertility.csv(3, [4, 16, 16])': 34.01809740066528,
 'Fertility.csv(3, [8, 4, 4])': 33.15605592727661,
 'Fertility.csv(3, [8, 4, 8])': 31.686266660690308,
 'Fertility.csv(3, [8, 4, 12])': 32.04465055465698,
 'Fert